**Abstract.**
I reached the point in my machine learning journey where I understood the basic concepts
(backpropagation, regularization, CNNs, RNNs, and transformers), but wanted to dig deeper.
The machine learning engineers that I knew suggested that I train a model for a problem I
found interesting. I selected learning regular expressions from examples as a toy problem.

# Generating training data

My plan for generating training data was:

1. Generate random regular expressions (regexes).
2. For each regex, generate some number of strings matching the regex.
3. For training data, let the $(x, y)$ input-output pairs be defined by:
   * $y$, a regex
   * $x$, a list of strings matching $y$, one per line
4. During training, reward if a forward pass through the network, starting with input $x$,
   produces an output $\hat{y}$ such that $y$ and $\hat{y}$ recognize the same strings.

## Imports

In [1]:
import os
import random
random.seed(42)

## Automata theory conversions

To work with regular expressions and finite automata, I started with
[Megha Bose's](https://github.com/Megha-Bose)
[Automata Theory Conversions](https://github.com/Megha-Bose/Automata-Theory-Conversions)
code. It wasn't licensed, so I asked her permission, and she gave me her
permission to use it.

## Generating random regexes

The following two problems did not seem like toy problems:

1. Creating a machine learning system capable of inferring long regexes.
2. Createing a machine learning system capable of inferring regexes with deep
   nestings of union and Kleene star operations.

Because I wanted a toy problem, I needed a way to generate short regexes having
shallow nestings. I settled on constructing a random parse tree having a
maximum depth.

In [2]:
# Union, Kleene star, concatenation, and grouping operations.
# '$' is special character that represents the empty string. 
# All other characters are considered symbols.
operators = ['+', '*', '.', '(', ')']


class OpType:
    """
    Constants for regular expression operator types.
    """
    SYMBOL = 1
    CONCAT = 2
    UNION  = 3
    KLEENE = 4


class ExpressionNode:
    """
    Node in an expression tree that represents the parse tree for a regular
    expression.
    """
    
    def __init__(self, op_type, value=None):
        self.op_type = op_type
        self.value = value
        self.left = None
        self.right = None

    def __str__(self):
        """
        Converts the node to a string in infix order, adding a lot of parens.
        """

        if self.op_type == OpType.SYMBOL:
            return self.value
        elif self.op_type == OpType.CONCAT:
            left = f'({self.left})' if self.left.op_type == OpType.UNION else str(self.left)
            right = f'({self.right})' if self.right.op_type == OpType.UNION else str(self.right)
            return left + right
        elif self.op_type == OpType.UNION:
            return f'{self.left}+{self.right}'
        elif self.op_type == OpType.KLEENE:
            if self.left.op_type == OpType.KLEENE:
                return str(self.left)
            elif self.left.op_type == OpType.SYMBOL:
                return str(self.left) + '*'
            else:
                return f'({self.left})*'
        else:
            raise Exception(f'unknown op type {self.op_type}')


def random_regex(max_depth=5, symbols="abcde", recurse_prob=0.95):
    """
    Generates a random regular expression.

    Args:
      max_depth: maximum depth of the parse tree
      symbols: allowed characters of the regular expression; must not contain 
               '$' nor any operator
      recurse_prob: probability that this method is invoked recursively

    Returns:
      expression tree representing the regular expression
    """

    can_recurse = max_depth > 1 and random.random() < recurse_prob

    op_type = random.randint(2, 4) if can_recurse else OpType.SYMBOL
    
    node = ExpressionNode(op_type)

    if op_type == OpType.SYMBOL:
        node.value = random.choice(symbols)
    else:
        node.left = random_regex(max_depth-1, symbols, recurse_prob)
        if op_type == OpType.CONCAT or op_type == OpType.UNION:
            node.right = random_regex(max_depth-1, symbols, recurse_prob)
        
    return node

In [3]:
for i in range(10):
    print(random_regex())

e*(bb+eb)((a+b)(c+b))*
de+a+e+(e+b)*+(b+c+d)(b+c+a*)
(e*+a+b)(be+b+d)(e*+e*)*
(bbb(e+a))*
e*
(c*(a+c)(c*+b))*
((cb)*+c*)(ea+a+e)b*d*
e
((e+d)*+eeaa)*
(d+b)e*(b+a)(c+d)a*


We've now generated random regexes, so we turn to generating strings matching a single regex.

## Generating strings matching a regex

To generate strings matching a regex, I first converted the regex into a non-deterministic
finite automaton (NFA), then followed random paths through the NFA from an initial node to
a terminal node.

### Step 1: rewrite regex (add concatenation, convert to postfix)

The first step in computing a regex to an NFA is to parse it. For this, it's useful to
rewrite the regex in a couple of ways:

1. Add concatenation symbols for convenience.
2. Convert the regex to postfix order.

In [4]:
def add_concat(regex):
    """
    Adds concatenation operators ('.') to the input regex for convenience.
    
    Args:
      regex: a string representing the regular expression
      
    Returns:
      a list representing the regular expression with added concatenation operators
    """
    
    global non_symbols

    result = []
    for i in range(len(regex) - 1):
        result.append(regex[i])
        if regex[i] not in operators:
            if regex[i + 1] not in operators or regex[i + 1] == '(':
                result += '.'
        elif regex[i] == ')' and regex[i + 1] == '(':
            result += '.'
        elif regex[i] == '*' and regex[i + 1] == '(':
            result += '.'
        elif regex[i] == '*' and regex[i + 1] not in operators:
            result += '.'
        elif regex[i] == ')' and regex[i + 1] not in operators:
            result += '.'

    result += regex[-1]
    return result

In [5]:
print(add_concat("ab*"))
print(add_concat("(ab)*"))
print(add_concat("(a+b)*c"))

['a', '.', 'b', '*']
['(', 'a', '.', 'b', ')', '*']
['(', 'a', '+', 'b', ')', '*', '.', 'c']


In [6]:
def compare_precedence(a, b):
    """
    Compares the precendence of the +, ., and * operators.
    
    Args:
      a: the first operator
      b: the second operator
      
    Returns:
      True iff a has precedence over b
    """
    p = ["+", ".", "*"]
    return p.index(a) > p.index(b)


def compute_postfix(regexp):
    """
    Converts an input regex to postfix order.
    
    Args:
      regex: list of regex operators and symbols
      
    Returns:
      list representing the regular expression in postfix order
    """
    
    stack = []
    result = []

    for c in regexp:
        if c not in operators or c == "*":
            result += c
        elif c == ")":
            while len(stack) > 0 and stack[-1] != "(":
                result += stack.pop()
            stack.pop()
        elif c == "(":
            stack.append(c)
        elif len(stack) == 0 or stack[-1] == "(" or compare_precedence(c, stack[-1]):
            stack.append(c)
        else:
            while len(stack) > 0 and stack[-1] != "(" and not compare_precedence(c, stack[-1]):
                result += stack.pop()
            stack.append(c)

    while len(stack) > 0:
        result += stack.pop()

    return result

In [7]:
print(compute_postfix(['a', '.', 'b']))
print(compute_postfix(['(', 'a', '.', 'b', ')']))
print(compute_postfix(['a', '.', 'b', '*']))
print(compute_postfix(['(', 'a', '.', 'b', ')', '*']))
print(compute_postfix(['(', 'a', '+', 'b', ')', '*', '.', 'c']))

['a', 'b', '.']
['a', 'b', '.']
['a', 'b', '*', '.']
['a', 'b', '.', '*']
['a', 'b', '+', '*', 'c', '.']


### Step 2: create expression tree from regex

Having converted the regex to postfix order, we create an expression tree from it. 

In [8]:
def make_expression_tree(regexp):
    """
    Makes an expression tree from a regex.
    
    Args:
      regex: a regular expression in postfix form
      
    Returns:
      an expression tree
    """
    
    stack = []

    for c in regexp:
        if c == "+":
            z = ExpressionNode(OpType.UNION)
            z.right = stack.pop()
            z.left = stack.pop()
            stack.append(z)
        elif c == ".":
            z = ExpressionNode(OpType.CONCAT)
            z.right = stack.pop()
            z.left = stack.pop()
            stack.append(z)
        elif c == "*":
            z = ExpressionNode(OpType.KLEENE)
            z.left = stack.pop() 
            stack.append(z)
        elif c == "(" or c == ")":
            continue  # just for safety; our input does not contain parens
        else:
            stack.append(ExpressionNode(OpType.SYMBOL, c))
    return stack[0]

In [9]:
print(make_expression_tree(['a', 'b', '.']))
print(make_expression_tree(['a', 'b', '*', '.']))
print(make_expression_tree(['a', 'b', '.', '*']))
print(make_expression_tree(['a', 'b', '+', '*', 'c', '.']))

ab
ab*
(ab)*
(a+b)*c


### Step 3: convert expression tree into a NFA

Having parsed the regex into a tree, we now convert the tree into a nondeterministic finite automaton (NFA).

We use `$` to denote the empty string; in textbooks $\epsilon$ normally denotes the empty string.

In [10]:
class NFAState:
    def __init__(self):
        self.next_state = {}

    def __str__(self):
        lines = ['NFAState ' + str(id(self))]
        for symbol, states in self.next_state.items():
            for state in states: 
                lines.append(f'\t{symbol}:\t{str(id(state))}')
        return '\n'.join(lines)

def compute_nfa(node):
    """
    Compute the NFA for an ExpressionNode.
    
    Args:
      node: an ExpressionNode
      
    Returns:
      the start and end NFAState for node
    """
    
    if node.op_type == OpType.CONCAT:
        left_nfa  = compute_nfa(node.left)
        right_nfa = compute_nfa(node.right)

        # '$' represents the empty string (usually denoted epsilon)
        left_nfa[1].next_state['$'] = [right_nfa[0]]
        return left_nfa[0], right_nfa[1]

    elif node.op_type == OpType.UNION:
        start = NFAState()
        end = NFAState()

        first_nfa = compute_nfa(node.left)
        second_nfa = compute_nfa(node.right)

        start.next_state['$'] = [first_nfa[0], second_nfa[0]]
        first_nfa[1].next_state['$'] = [end]
        second_nfa[1].next_state['$'] = [end]

        return start, end

    elif node.op_type == OpType.KLEENE:
        start = NFAState()
        end = NFAState()

        starred_nfa = compute_nfa(node.left)

        start.next_state['$'] = [starred_nfa[0], end]
        starred_nfa[1].next_state['$'] = [starred_nfa[0], end]
        
        return start, end

    elif node.op_type == OpType.SYMBOL:
        start = NFAState()
        end = NFAState()
        start.next_state[node.value] = [end]
        return start, end
    
    else:
        raise Exception(f'unknown op type {node.op_type}')

### Putting steps 1-3 together

We now wrap up steps 1-3 in a function that takes a regex as argument and returns an NFA.

In [11]:
def regex_to_nfa(regex):
    """
    Compute the NFA for a regular expression.
    
    Args:
      regex: a regular expression
    
    Returns:
      the start and end NFAState
    """
    
    concat = add_concat(regex)
    postfix = compute_postfix(concat)
    tree = make_expression_tree(postfix)
    start, end = compute_nfa(tree)
    return start, end

In [12]:
print(regex_to_nfa("ab*"))
print(regex_to_nfa("(ab)*"))
print(regex_to_nfa("(a+b)*c"))
print(regex_to_nfa(str(random_regex(6))))

(<__main__.NFAState object at 0x7f949c365c30>, <__main__.NFAState object at 0x7f949c3651b0>)
(<__main__.NFAState object at 0x7f949c366f50>, <__main__.NFAState object at 0x7f949c365ae0>)
(<__main__.NFAState object at 0x7f949c365270>, <__main__.NFAState object at 0x7f949c3670d0>)
(<__main__.NFAState object at 0x7f949c365fc0>, <__main__.NFAState object at 0x7f949c365c90>)


### Step 4: random paths through an NFA

Starting with an NFA, we've generated a regex. The next step is to generate strings matching
the regex from the NFA. To do that, we follow a random path through the NFA from the initial
node. We build the sequence of edge labels we see along the path traveled so far. When we reach
the terminal node, we output the edge labels we saw.

In [13]:
def nfa_path(start, end, max_steps):
    """
    Follow a random path through an NFA, accumulating edge labels.
    
    Args:
      start: start NFAState
      end: end NFAState
      
    returns:
      ':' followed by edge labels, or an empty string if no path
      to end state of at most max_steps steps was found
    """
    
    step = 0
    state = start
    found_path = False
    labels = []
    
    while step < max_steps:
        if state == end:
            found_path = True
            break
            
        # select random label
        i = random.randint(0, len(state.next_state) - 1)
        label = list(state.next_state)[i]
        if label != '$':
            labels.append(label)
        
        # select random next state
        i = random.randint(0, len(state.next_state[label]) - 1)
        state = state.next_state[label][i]
    
    if found_path:
        return ':' + ''.join(labels)
    else:
        return ''

In [14]:
regexes = ['ab*', '(ab)*', '(a+b)*c*', str(random_regex(4, 'abc'))]

for regex in regexes:
    print('regex', regex)
    start, end = regex_to_nfa(regex)
    for i in range(0, 10):
        print(nfa_path(start, end, 8))
    print('-----')

regex ab*
:abbbbb
:abbb
:a
:a
:ab
:ab
:abb
:abbbb
:a
:a
-----
regex (ab)*
:
:
:
:ab
:
:
:
:
:ab
:
-----
regex (a+b)*c*
:
:
:
:
:acc
:acc
:abc
:b
:c
:
-----
regex ba+b+c*+c
:
:ccc
:c
:b
:c
:c
:c
:c
:c
:c
-----


### Step 5: create training data

We now generate training data as files in a data directory. I decided to start
simple, with regexes having a maximum depth of 4.

In [15]:
def create_training_data(files, examples_per_file,
                         max_depth=4, symbols="abcdefghijklmnop",
                         recurse_prob=0.95):
    """
    Generate training data. Each file contains a regex and some strings
    matching the regex.

    Args:
      files: number of files to create
      examples_per_file: number of matching strings per file
      max_depth: see random_regex()
      symbols: see random_regex()
      recurse_prob: see random_regex()
    """
    
    os.mkdir('data')
    
    for i in range(files):
        with open(f'data/{i:06}.regex', 'x') as file:
            regex = str(random_regex(max_depth=max_depth, symbols=symbols,
                                     recurse_prob=recurse_prob))

            start, end = regex_to_nfa(regex)

            # choose a reasonable number of steps through the NFA.
            # empirically allowing max_steps = 4 * examples is enough
            # to frequently reach an accepting state
            max_steps = examples_per_file * 4
            
            for i in range(examples_per_file):
                path = ''
                # repeat until we find a path to an accepting state:
                while not path.startswith(':'):
                    path = nfa_path(start, end, max_steps)

                file.write(f'{path}\n')

            # indicate that no more examples follow with the prompt '>'
            file.write('>\n')

            # finally, write the regex
            file.write(f'{regex}\n')

In [16]:
!rm -r data

I decided to generate one million regexes worth of training data. Empirically,
I found that when using the function `nfa_path()`, there were at most 900
unique strings generated per regex. It seemed reasonable to generate some
repeats, both to allow training to guess the structure of the regex, and to
allow `nfa_path()` to cover the NFA's structure. So I chose to generate 10,000
examples per file.

In [17]:
files = 1_000_000
examples_per_file = 10_000
create_training_data(files, examples_per_file)

And there we have it -- one million training files!

## Determining if two regexes are equivalent

We will need to define our loss function such that it rewards a prediction
$\hat{y}$ if $y$ and $\hat{y}$ recognize the same strings. How do we tell
whether two regexes recognize the same strings? The classic algorithm
has us do the following:

1. Convert $y$ and $\hat{y}$ to NFAs $N$ and $\hat{N}$.
2. Convert $N$ and $\hat{N}$ to deterministic finite automata (DFAs) $D$ and
   $\hat{D}$.
3. Minimize $D$ and $\hat{D}$, obtaining DFAs $M$ and $\hat{M}$.
4. Check whether $M$ and $\hat{M}$ are identical modulo renaming of states.

In this section, we develop the code necessary for steps 2-4. Step 1 was
already covered when we converted a regex to an NFA for the purpose of
generating strings matching the regex.

### Step 1: convert regex to NFA

Automata-Theory-Constructions already had code for this, which I modified a little (see `regex_to_nfa()`).

### Step 2: NFA to DFA

This is a classic subset construction algorithm from the theory of finiste
automata. We first write a function to compute the set of all states reachable
from a given state by following only transitions labeled with $\epsilon$ (`$`
in our code).

In [18]:
def epsilon_closure(states):
    """
    Compute the set of all states that can be reached from the given states by
    following only epsilon ($) transitions. 

    Args:
      states: iterable of NFAState
      visited: set of already visited states

    Returns:
      frozenset of states reachable from states by following only epsilon ($)
      transitions.
    """
    
    def helper(states, visited):    
        for state in states:
            if state not in visited:
                try:
                    helper(state.next_state['$'], visited)
                except KeyError:
                    pass
                visited.add(state)

    visited = set()
    helper(states, visited)
    
    return frozenset(visited)

In [19]:
# set up transitions as follows.
# all transitions are epsilon transitions except d -> e
# a -> b -> d -> e
# |         ^
# +--> c ---+
def test_epsilon_closure():
    a = NFAState()
    b = NFAState()
    c = NFAState()
    d = NFAState()
    e = NFAState()

    a.next_state['$'] = [b, c]
    b.next_state['$'] = [d]
    c.next_state['$'] = [d]
    d.next_state['x'] = [e]
    
    assert epsilon_closure([a]) == {a, b, c, d}

test_epsilon_closure()

Next we write a function that converts an NFA to a DFA. The function uses
depth-first search. Given a set of NFA states constituting a DFA state, and a
symbol, the search finds the NFA states reachable from the DFA state by
following the symbol.

This is similar to the classic subset construction, except that we don't
compute the power set of NFA states up front, and instead let search find the
reachable DFA states for us.

In [34]:
class DFAState:
    def __init__(self, nfa_states, end_nfa_state):
        self.nfa_states = nfa_states
        self.next_state = {}
        self.is_accepting = any(end_nfa_state == s for s in nfa_states)

    def __str__(self):
        lines = ['DFAState ' + str(id(self))]
        if self.is_accepting:
            lines.append('accepting')
        for symbol, state in self.next_state.items():
            lines.append(f'\t{symbol}:\t{str(id(state))}')
        return '\n'.join(lines)

class DFA:
    def __init__(self, states, start):
        self.states = states
        self.start = start

    def __str__(self):
        lines = []
        for state in self.states:
            lines.append('-----')
            if state == self.start:
                lines.append('start state')
            if len(state.nfa_states) == 0:
                lines.append('empty state')
            lines.append(str(state))
        return '\n'.join(lines)


def nfa_to_dfa(start, end, symbols):
    """
    Convert an NFA to a DFA.

    Args:
      start: the start NFAState
      end: the end NFAState
      symbols: the alphabet of the NFA

    Returns:
      a DFA
    """

    start_state = DFAState(epsilon_closure([start]), end)  
    frontier = [start_state]
    visited = set()
    dfa_states = {} # key: set of NFAState, value: DFAState
    
    while len(frontier) > 0:
        state = frontier.pop()

        if state in visited:
            continue

        for symbol in symbols:
            # find NFA states reachable from state by following symbol
            reachable = set()

            for nfa_state in state.nfa_states:
                try:
                    reachable.update(nfa_state.next_state[symbol])
                except KeyError:
                    pass

            reachable = epsilon_closure(reachable)
            
            if reachable in dfa_states:
                next = dfa_states[reachable]
            else:
                next = DFAState(reachable, end)
                dfa_states[reachable] = next
            
            state.next_state[symbol] = next
            
            if next not in visited:
                frontier.append(next)
        
        visited.add(state)
    
    return DFA(visited, start_state) 

In [38]:
regexes = ['a', 'b*']
symbols = ['a', 'b']

for i in range(len(regexes)):
    start, end = regex_to_nfa(regexes[i])
    dfa = nfa_to_dfa(start, end, symbols[i])
    print(f'DFA for {regexes[i]}:')
    print(str(dfa))

DFA for a:
-----
start state
DFAState 140274275416192
	a:	140274278024624
-----
DFAState 140274278024624
accepting
	a:	140274278024768
-----
empty state
DFAState 140274278024768
	a:	140274278024768
DFA for b*:
-----
start state
DFAState 140274275417344
accepting
	b:	140274276305600
-----
DFAState 140274276305600
accepting
	b:	140274276305600


### Step 3: minimize DFA

TODO

### Step 4: check whether DFAs are identical

TODO